In [17]:
load('Imports/Utility.sage', 'Imports/Point.sage', 'Imports/Line.sage', 'Imports/TritangentPlane.sage', 'Imports/Group.sage', 'Imports/Cubic.sage')
import multiprocessing as mp
import pickle

In [18]:
P.<x,y,z,t,b,c,d,e,f,l,m> = PolynomialRing(QQ)
cubic_new = e*f*(2*x^2*y-2*x*y^2+x*z^2-x*z*t-y*t^2+y*z*t)+b*c*(x-t)*(x*z+y*t)+c*c*(z+t)*(y*t-x*z)+d*c*(y-z)*(x*z+y*t)+(e+f)*c*(x-y)*(y*t-x*z)
sing_cubics = (-1) * (-c + f) * (-c + e) * c * (c + f) * (c + e) * (-e + f)^2 * (-c*d + c*f + e*f) * (-c*d + c*e + e*f) * (-c^2 - c*d + e*f) * (b*c - c*f + e*f) * (b*c - c*e + e*f) * (b*c - c*d + 2*e*f) * (b*c - c^2 + e*f) * (b*c^2 + c^2*d + b*c*f - 2*c^2*f - c*d*f + 2*e*f^2) * (b*c^2 + c^2*d + b*c*e - 2*c^2*e - c*d*e + 2*e^2*f) * (-b*c^3 - 2*b*c^2*d + c^3*d + b*c^2*e + c^2*d*e + b*c^2*f + c^2*d*f + 3*b*c*e*f - 4*c^2*e*f - 3*c*d*e*f + 4*e^2*f^2)

In [19]:
# Define cubic SE3, find and classify the lines and compute the 45 tritangent planes

In [20]:
line = Line([y, z])
general_cubic = Cubic(cubic_new, line, sing_cubics)
SE3 = general_cubic.subs({b:-(c*c+e*f)/c, d:(3*e*f - c^2 + c*f + c*e)/(2*c)})

In [21]:
# Find simmetries

In [22]:
%time adm_SE3 = SE3.find_admissible_projectivities()
len(adm_SE3)

CPU times: user 504 ms, sys: 92.4 ms, total: 596 ms
Wall time: 2.03 s


108

In [23]:
%time simm_SE3 = SE3.find_simmetries(adm_SE3)
len(simm_SE3)

CPU times: user 348 ms, sys: 129 ms, total: 477 ms
Wall time: 603 ms


6

In [24]:
# Study how the simmetries permute the Eckardt points

In [25]:
Eck_points_perms = []
for proj in simm_SE3:
    Eck_points_perms.append(Permutation(SE3.apply_proj_to_eck(proj)).to_permutation_group_element())
Eck_points_perms_group = PermutationGroup(Eck_points_perms)
Eck_points_perms_group.order(), Eck_points_perms_group.structure_description()

(6, 'S3')

In [26]:
# First generator fixes A, permutes C, F. Second permutes A -> C -> F -> A
gens = Eck_points_perms_group.gens_small()
indices = [Eck_points_perms.index(gen) for gen in gens]

In [27]:
# Study how the simmetries permute the 27 lines

In [28]:
lines_perms = []
for simm in simm_SE3:
    lines_perms.append(Permutation(SE3.apply_proj_to_lines(simm)).to_permutation_group_element())
lines_perms_group = PermutationGroup(lines_perms)
lines_perms_group.order(), lines_perms_group.structure_description()

(6, 'S3')

In [29]:
gens_lines = [lines_perms[index] for index in indices]
[get_permuted_extended_L_set(perm) for perm in gens_lines], gens_lines

([('E1', 'G4', 'F24', 'F13', 'F34', 'F45'),
  ('F13', 'F24', 'G4', 'E1', 'E4', 'F16')],
 [(2,19)(3,22)(4,7)(5,25)(6,26)(8,13)(9,14)(11,16)(12,17)(18,27)(20,24)(21,23),
  (1,14,9)(2,10,19)(3,4,13)(5,17,21)(6,16,20)(7,22,8)(11,26,24)(12,25,23)(15,27,18)])

In [30]:
PermutationGroup(gens_lines).is_isomorphic(Eck_points_perms_group)

True

In [31]:
# Find possible conditions to obtain a subfamily with a larger simmetry group

In [32]:
%time SE3.find_conditions_for_subfamilies(adm_SE3, simm_SE3)

CPU times: user 7.67 s, sys: 65.5 ms, total: 7.74 s
Wall time: 7.91 s


[]